In [1]:
import pickle
import numpy as np
import torch
import json

def preprocess_item_embeddings(embedding_file, output_file, device='cpu'):
    """
    embedding_file: str, 원래 item_embedding_normalized.pickle 파일 경로
    output_file: str, 출력 파일 (예: item_embedding_normalized_revised.pickle)
    device: str, device (주로 CPU로 처리 후 저장)
    
    pickle 파일은 { item_id: embedding (numpy array) } 형태라고 가정.
    
    전처리:
      - 모든 item id를 int로 변환 후, 유니크하게 정렬
      - 0번 인덱스를 padding으로 사용할 것이므로, mapping: {원본_id: new_index} 를 새롭게 만듦, new_index는 1부터 시작
      - 각 embedding을 torch.tensor로 변환하고, index 0에는 0벡터 삽입
      - 최종적으로, { 'embedding_tensor': tensor, 'id2index': mapping } 을 저장
    """
    # pickle 파일 로드
    with open(embedding_file, 'rb') as f:
        embedding_dict = pickle.load(f)
    
    # 모든 item id 추출 (key가 문자열일 수 있으므로 int로 변환)
    raw_ids = [int(key) for key in embedding_dict.keys()]
    sorted_ids = sorted(set(raw_ids))

    # mapping: 0번을 padding 용으로 사용하므로 new_index는 1부터 시작
    id2index = {orig_id: idx+1 for idx, orig_id in enumerate(sorted_ids)}
    num_items = len(sorted_ids)
    print(f"Number of unique items: {num_items}, Largest item id: {max(sorted_ids)}, Smallest item id: {min(sorted_ids)}")

    # embedding 차원 확인 (모든 embedding은 동일하다고 가정)
    sample_embedding = next(iter(embedding_dict.values()))
    embedding_dim = sample_embedding.shape[0] if isinstance(sample_embedding, np.ndarray) else len(sample_embedding)
    
    # index 0: padding 벡터 (0으로 채움)
    embedding_list = [torch.zeros(embedding_dim, dtype=torch.float32)]
    
    # 정렬된 id 순으로 embedding을 torch.tensor화
    for orig_id in sorted_ids:
        emb_np = embedding_dict[str(orig_id)] if str(orig_id) in embedding_dict else embedding_dict[orig_id]
        emb_tensor = torch.tensor(emb_np, dtype=torch.float32)
        embedding_list.append(emb_tensor)
    
    # stacking하여 tensor로 만들기: shape [num_items+1, embedding_dim]
    embedding_tensor = torch.stack(embedding_list, dim=0)
    
    # 결과 저장: CPU 상의 tensor로 저장
    output = {
        'embedding_tensor': embedding_tensor,
        'id2index': id2index
    }
    #with open(output_file, 'wb') as f:
        #pickle.dump(output, f)
    
    print(f"Preprocessed item embeddings saved to {output_file}")
    return output

def preprocess_candidate_sets_revised(candidate_file, output_file, id2index):
    """
    Args:
      candidate_file: str, 원본 candidate_sets.npz 파일 경로.
      output_file: str, 저장할 _revised candidate file 경로 (예: candidate_sets_revised.npz)
      id2index: dict, 원본 item id -> new index (이미 전처리 단계에서 생성된 mapping)
    
    전처리:
      - npz 파일에서 candidate_keys와 candidate_values를 로드합니다.
      - 각 후보 리스트를 재매핑하여, 원본 candidate 개수를 그대로 사용합니다.
      - 만약 후보 정보가 부족하면 자기 자신으로 채움.
      
    Returns:
      candidate_tensor: [N+1, candidate_num] long tensor, 여기서 N+1은 전체 item 수 (패딩 포함)
                        candidate_num은 원본 파일에 들어있는 후보 개수.
    """
    data = np.load(candidate_file)
    candidate_keys = data["keys"]  # 원본 candidate key: 보통 item id (문자열 또는 int)
    candidate_values = data["values"]  # 각 원소는 후보 리스트 (예: list of int)

    # 임시 dict 생성: 원본 id -> candidate list
    temp_candidate_dict = {}
    for i, k in enumerate(candidate_keys):
        orig_id = int(k)
        # 여기서는 후보 개수를 변경하지 않고 그대로 사용합니다.
        cand_list = candidate_values[i].tolist()
        # 만약 부족하다면, 자기 자신으로 채움
        # (이 부분은 상황에 따라 조정 가능)
        temp_candidate_dict[orig_id] = cand_list

    # 전체 아이템 수: id2index 매핑에 기반 (padding index 0 포함: 전체 개수 = len(id2index)+1)
    N = len(id2index) + 1
    # 후보 수: 원본 candidate list 중 하나의 길이를 사용 (모두 동일하다고 가정)
    # 만약 item마다 다를 경우, 최대 길이를 사용할 수 있음.
    candidate_num = max(len(lst) for lst in candidate_values)
    
    # candidate_tensor 초기화: [N, candidate_num]
    candidate_tensor = torch.empty((N, candidate_num), dtype=torch.long)
    # index 0은 padding, 보통 자기 자신 혹은 0 값으로 처리 - 여기서는 0으로 채움.
    candidate_tensor[0] = torch.zeros(candidate_num, dtype=torch.long)
    
    for orig_id, new_idx in id2index.items():
        if orig_id in temp_candidate_dict:
            orig_candidates = temp_candidate_dict[orig_id]  # list of original candidate ids
            # 재매핑: 각 candidate id를 id2index를 통해 new index로 변환
            new_candidates = []
            for cand in orig_candidates:
                new_candidates.append(id2index.get(cand, new_idx))  # 매핑이 없으면 자기 자신
            # 만약 후보 수가 candidate_num보다 작으면, 부족한 부분을 자기 자신(new_idx)으로 채움
            if len(new_candidates) < candidate_num:
                new_candidates += [new_idx] * (candidate_num - len(new_candidates))
            else:
                new_candidates = new_candidates[:candidate_num]
            candidate_tensor[new_idx] = torch.tensor(new_candidates, dtype=torch.long)
        else:
            # 후보 정보가 없는 경우, 자기 자신으로 채움
            candidate_tensor[new_idx] = torch.full((candidate_num,), new_idx, dtype=torch.long)
    
    # 저장: np.savez 사용. tensor는 numpy array로 변환.
    #np.savez(output_file, candidate_tensor=candidate_tensor.cpu().numpy())
    print(f"Preprocessed candidate sets saved to {output_file}")
    return candidate_tensor

# 예시 인자들 (실제 파일 경로와 candidate_size를 적절히 수정)
folder = "Retail_Rocket/"
item_embedding_file = folder + "item_embedding_normalized.pickle"
candidate_file = folder + "candidate_sets.npz"
revised_embedding_file = folder + "item_embedding_normalized_revised.pickle"
revised_candidate_file = folder + "candidate_sets_revised.npz"
candidate_size = 128   # 예시 값

device = 'cpu'  # 전처리 단계는 CPU에서 진행한 후, 나중에 학습 시 device에 올리면 됩니다.

In [2]:
# 1. item embeddings 전처리
embedding_output = preprocess_item_embeddings(item_embedding_file, revised_embedding_file, device)
id2index = embedding_output['id2index']

Number of unique items: 417053, Largest item id: 466866, Smallest item id: 0
Preprocessed item embeddings saved to Retail_Rocket/item_embedding_normalized_revised.pickle


In [3]:
# 2. candidate sets 전처리
candidate_tensor = preprocess_candidate_sets_revised(candidate_file, revised_candidate_file, id2index)

Preprocessed candidate sets saved to Retail_Rocket/candidate_sets_revised.npz


In [ ]:
import json

def preprocess_interactions(interaction_file, output_file, id2index):
    """
    interaction_file: str, 원래 interaction.json 파일 경로.
    output_file: str, 전처리된 interaction 파일 경로 (예: interactions_revised.json)
    id2index: dict, 원래 item id -> new index mapping
    
    각 interaction 내의 item id를 id2index를 사용해 변환합니다.
    만약 변환할 수 없는 id가 있으면, -1 또는 0 (패딩)으로 처리합니다.
    """
    with open(interaction_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # data 형식이 {"data": {user_id: sessions, ...}, "index": ... } 등이라고 가정
    trash_item_user = set()
    total_interaction = 0
    total_trash_interaction = 0
    for user_id, sessions in data["data"].items():
        for i, session in enumerate(sessions):
            for j, interaction in enumerate(session):
                # interaction은 [item_id, timestamp, add_info]
                orig_id = interaction[0]
                # 만약 orig_id가 -1 (패딩)이면 그대로 유지, 아니면 변환
                if orig_id == -1:
                    new_id = -1
                else:
                    # int 변환 후 mapping 적용. 만약 mapping에 없으면 0 (또는 -1) 처리
                    total_interaction += 1
                    try:
                        new_id = id2index[int(orig_id)]
                    except Exception as e:
                        trash_item_user.add(user_id)
                        new_id = -1
                        total_trash_interaction += 1
                data["data"][user_id][i][j][0] = new_id
    print(f"Number of unique users: {len(data['data'])}")
    print(len(trash_item_user), "users have trash items.")
    """with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)"""
    print(f"Preprocessed interactions saved to {output_file}")
    print(f"Total interactions: {total_interaction}, Total trash interactions: {total_trash_interaction}")
    return output_file

# 사용 예시:
# 이미 preprocess_item_embeddings를 통해 id2index를 생성했다면,
# 예: embedding_output = preprocess_item_embeddings(embedding_file, revised_embedding_file)
#      id2index = embedding_output['id2index']
interaction_file = folder + "interactions.json"
output_file = folder + "interactions_revised.json"
preprocess_interactions(interaction_file, output_file, id2index)

Number of unique users: 96057
14688 users have trash items.
Preprocessed interactions saved to Retail_Rocket/interactions_revised.json


'Retail_Rocket/interactions_revised.json'